In [ ]:
import torch                                      # root package
from torch.utils.data import Dataset, DataLoader  # data 
import torch.autograd as autograd                 # computation graph
from torch import Tensor                          # tensor node in the computation graph
import torch.nn as nn                             # neural networks
import torch.nn.functional as F                   # layers, activations and more
import torch.optim as optim  
InteractiveShell.ast_node_interactive = "all"
warnings.filterwarnings("ignore")

# MobileNet
 - Designed to be small and low latency.
  
 - Trainable on mobile devices.
 - Standard multi-channel convolutions expensive opertation 
 - Where more than 1 filter used (with multi-channel convolutions), favourable to replace with **depth-wise** followed by **point-wise** convolutions. (layers together are called **depth-wise separable** filters)
 - MobileNet use 3 depth-wise separable layers
 

In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)
model.eval()

# AlexNet

In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
model.eval()

# VGGNet 

In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg11', pretrained=True)
model.eval()

# Inception

In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=True)
model.eval()

# ResNet